# Services


ClusterIP services provide a stable network identity for a set of pods.

First let's deploy an NGINX farm:

In [1]:
kubectl apply -f nginx.yaml

deployment.apps/nginx-deployment created


Let's inspect this deployment:


In [2]:
kubectl describe deployment nginx-deployment

Name:                   nginx-deployment
Namespace:              default
CreationTimestamp:      Thu, 27 Jun 2019 10:31:26 +0000
Labels:                 <none>
Annotations:            deployment.kubernetes.io/revision: 1
                        kubectl.kubernetes.io/last-applied-configuration:
                          {"apiVersion":"apps/v1","kind":"Deployment","metadata":{"annotations":{},"name":"nginx-deployment","namespace":"default"},"spec":{"replica...
Selector:               app=nginx
Replicas:               2 desired | 2 updated | 2 total | 2 available | 0 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  25% max unavailable, 25% max surge
Pod Template:
  Labels:  app=nginx
  Containers:
   nginx:
    Image:        nginx:1.7.9
    Port:         80/TCP
    Host Port:    0/TCP
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Conditions:
  Type           Status  Reason
  ----           ------  ------
  Ava

Listing the pods:


In [5]:
kubectl get pods -l app=nginx
kubectl get pods -l app=nginx -o yaml | grep ' podIP:'


NAME                               READY   STATUS    RESTARTS   AGE
nginx-deployment-6dd86d77d-qwn2k   1/1     Running   0          2m3s
nginx-deployment-6dd86d77d-zzgtv   1/1     Running   0          2m3s
    podIP: 10.200.165.70
    podIP: 10.200.165.71


Creating a service:

In [7]:
kubectl apply -f nginx-service.yaml

service/my-nginx configured


In [8]:
kubectl get svc my-nginx


NAME       TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)   AGE
my-nginx   ClusterIP   10.100.200.79   <none>        80/TCP    39s


In [9]:
kubectl describe svc my-nginx


Name:              my-nginx
Namespace:         default
Labels:            run=my-nginx
Annotations:       kubectl.kubernetes.io/last-applied-configuration:
                     {"apiVersion":"v1","kind":"Service","metadata":{"annotations":{},"labels":{"run":"my-nginx"},"name":"my-nginx","namespace":"default"},"spe...
Selector:          app=nginx
Type:              ClusterIP
IP:                10.100.200.79
Port:              <unset>  80/TCP
TargetPort:        80/TCP
Endpoints:         10.200.165.70:80,10.200.165.71:80
Session Affinity:  None
Events:            <none>


A service fundamentally is an mapping of a Cluster IP in the service network to a set of endpoints on the POD network:


In [1]:
kubectl describe svc my-nginx | grep IP:
kubectl get ep my-nginx


IP:                10.100.200.79
NAME       ENDPOINTS                           AGE
my-nginx   10.200.165.70:80,10.200.165.71:80   6m49s


Paste the following in a terminal:

```
kubectl run --generator=run-pod/v1  curl --image=radial/busyboxplus:curl -i --tty
nslookup my-nginx     # each service gets a DNS entry per its name
curl http://my-nginx  # should output the default web page
exit
```




Let's explore what's on the cluster node:

```
bosh ssh worker
sudo su
iptables -L -t nat -n -v 
```

This should output something similar to the following:

```
Chain PREROUTING (policy ACCEPT 1 packets, 40 bytes)
 pkts bytes target     prot opt in     out     source               destination
 1268 73753 cali-PREROUTING  all  --  *      *       0.0.0.0/0            0.0.0.0/0            /* cali:6gwbT8clXdHdC1b1 */
 1271 74892 KUBE-SERVICES  all  --  *      *       0.0.0.0/0            0.0.0.0/0            /* kubernetes service portals */

Chain INPUT (policy ACCEPT 1 packets, 40 bytes)
 pkts bytes target     prot opt in     out     source               destination

Chain OUTPUT (policy ACCEPT 14 packets, 844 bytes)
 pkts bytes target     prot opt in     out     source               destination
23905 1445K cali-OUTPUT  all  --  *      *       0.0.0.0/0            0.0.0.0/0            /* cali:tVnHkvAo15HuiPy0 */
24019 1452K KUBE-SERVICES  all  --  *      *       0.0.0.0/0            0.0.0.0/0            /* kubernetes service portals */

Chain POSTROUTING (policy ACCEPT 14 packets, 844 bytes)
 pkts bytes target     prot opt in     out     source               destination
23928 1446K cali-POSTROUTING  all  --  *      *       0.0.0.0/0            0.0.0.0/0            /* cali:O3lYWMrLQYEMJtB5 */
24093 1457K KUBE-POSTROUTING  all  --  *      *       0.0.0.0/0            0.0.0.0/0            /* kubernetes postrouting rules */

Chain KUBE-MARK-DROP (0 references)
 pkts bytes target     prot opt in     out     source               destination
    0     0 MARK       all  --  *      *       0.0.0.0/0            0.0.0.0/0            MARK or 0x8000

Chain KUBE-MARK-MASQ (18 references)
 pkts bytes target     prot opt in     out     source               destination
    0     0 MARK       all  --  *      *       0.0.0.0/0            0.0.0.0/0            MARK or 0x4000

Chain KUBE-NODEPORTS (1 references)
 pkts bytes target     prot opt in     out     source               destination
    0     0 KUBE-MARK-MASQ  tcp  --  *      *       0.0.0.0/0            0.0.0.0/0            /* kube-system/kubernetes-dashboard: */ tcp dpt:30249
    0     0 KUBE-SVC-XGLOHA7QRQ3V22RZ  tcp  --  *      *       0.0.0.0/0            0.0.0.0/0            /* kube-system/kubernetes-dashboard: */ tcp dpt:30249

Chain KUBE-POSTROUTING (1 references)
 pkts bytes target     prot opt in     out     source               destination
    0     0 MASQUERADE  all  --  *      *       0.0.0.0/0            0.0.0.0/0            /* kubernetes service traffic requiring SNAT */ mark match 0x4000/0x4000

Chain KUBE-SEP-2Q2G5JSM4U6DJ5CG (1 references)
 pkts bytes target     prot opt in     out     source               destination
    0     0 KUBE-MARK-MASQ  all  --  *      *       10.200.165.65        0.0.0.0/0
    0     0 DNAT       udp  --  *      *       0.0.0.0/0            0.0.0.0/0            udp to:10.200.165.65:53

Chain KUBE-SEP-7IVKYNOTO223MVHZ (1 references)
 pkts bytes target     prot opt in     out     source               destination
    0     0 KUBE-MARK-MASQ  all  --  *      *       10.200.165.65        0.0.0.0/0
    0     0 DNAT       tcp  --  *      *       0.0.0.0/0            0.0.0.0/0            tcp to:10.200.165.65:53

Chain KUBE-SEP-AOFVCMQOP4BSPNKH (1 references)
 pkts bytes target     prot opt in     out     source               destination
    0     0 KUBE-MARK-MASQ  all  --  *      *       10.200.165.70        0.0.0.0/0
    0     0 DNAT       tcp  --  *      *       0.0.0.0/0            0.0.0.0/0            tcp to:10.200.165.70:80

Chain KUBE-SEP-FQ4F6LUEKDKKCOMF (1 references)
 pkts bytes target     prot opt in     out     source               destination
    0     0 KUBE-MARK-MASQ  all  --  *      *       10.200.165.68        0.0.0.0/0
    0     0 DNAT       tcp  --  *      *       0.0.0.0/0            0.0.0.0/0            tcp to:10.200.165.68:443

Chain KUBE-SEP-JKMNWT3R5EVONNDL (1 references)
 pkts bytes target     prot opt in     out     source               destination
    0     0 KUBE-MARK-MASQ  all  --  *      *       10.200.165.67        0.0.0.0/0
    0     0 DNAT       udp  --  *      *       0.0.0.0/0            0.0.0.0/0            udp to:10.200.165.67:53

Chain KUBE-SEP-MWWFI2FSQNNTZPVG (1 references)
 pkts bytes target     prot opt in     out     source               destination
    0     0 KUBE-MARK-MASQ  all  --  *      *       10.200.165.66        0.0.0.0/0
    0     0 DNAT       tcp  --  *      *       0.0.0.0/0            0.0.0.0/0            tcp to:10.200.165.66:53

Chain KUBE-SEP-O4OJPHL6SYITWREJ (1 references)
 pkts bytes target     prot opt in     out     source               destination
    0     0 KUBE-MARK-MASQ  all  --  *      *       10.0.0.23            0.0.0.0/0
    0     0 DNAT       tcp  --  *      *       0.0.0.0/0            0.0.0.0/0            tcp to:10.0.0.23:8443

Chain KUBE-SEP-ORNXWSX5YDVWULHU (1 references)
 pkts bytes target     prot opt in     out     source               destination
    0     0 KUBE-MARK-MASQ  all  --  *      *       10.200.165.67        0.0.0.0/0
    0     0 DNAT       tcp  --  *      *       0.0.0.0/0            0.0.0.0/0            tcp to:10.200.165.67:53

Chain KUBE-SEP-QVYN2TD7L6IA7X3K (1 references)
 pkts bytes target     prot opt in     out     source               destination
    0     0 KUBE-MARK-MASQ  all  --  *      *       10.200.165.66        0.0.0.0/0
    0     0 DNAT       udp  --  *      *       0.0.0.0/0            0.0.0.0/0            udp to:10.200.165.66:53

Chain KUBE-SEP-VAZHZYTNC3JEIHVH (1 references)
 pkts bytes target     prot opt in     out     source               destination
    0     0 KUBE-MARK-MASQ  all  --  *      *       10.200.165.71        0.0.0.0/0
    0     0 DNAT       tcp  --  *      *       0.0.0.0/0            0.0.0.0/0            tcp to:10.200.165.71:80

Chain KUBE-SEP-XXKY7ZZPXQPQ7VS6 (1 references)
 pkts bytes target     prot opt in     out     source               destination
    0     0 KUBE-MARK-MASQ  all  --  *      *       10.200.165.69        0.0.0.0/0
    0     0 DNAT       tcp  --  *      *       0.0.0.0/0            0.0.0.0/0            tcp to:10.200.165.69:8443

Chain KUBE-SERVICES (2 references)
 pkts bytes target     prot opt in     out     source               destination
    0     0 KUBE-MARK-MASQ  tcp  --  *      *      !10.200.0.0/16        10.100.200.2         /* kube-system/kube-dns:dns-tcp cluster IP */ tcp dpt:53
    0     0 KUBE-SVC-ERIFXISQEP7F7OF4  tcp  --  *      *       0.0.0.0/0            10.100.200.2         /* kube-system/kube-dns:dns-tcp cluster IP*/ tcp dpt:53
    0     0 KUBE-MARK-MASQ  udp  --  *      *      !10.200.0.0/16        10.100.200.2         /* kube-system/kube-dns:dns cluster IP */ udp dpt:53
    0     0 KUBE-SVC-TCOU7JCQXEZGVUNU  udp  --  *      *       0.0.0.0/0            10.100.200.2         /* kube-system/kube-dns:dns cluster IP */ udp dpt:53
    0     0 KUBE-MARK-MASQ  tcp  --  *      *      !10.200.0.0/16        10.100.200.66        /* kube-system/metrics-server: cluster IP */ tcp dpt:443
    0     0 KUBE-SVC-LC5QY66VUV2HJ6WZ  tcp  --  *      *       0.0.0.0/0            10.100.200.66        /* kube-system/metrics-server: cluster IP */ tcp dpt:443
    0     0 KUBE-MARK-MASQ  tcp  --  *      *      !10.200.0.0/16        10.100.200.177       /* kube-system/kubernetes-dashboard: cluster IP */ tcp dpt:443
    0     0 KUBE-SVC-XGLOHA7QRQ3V22RZ  tcp  --  *      *       0.0.0.0/0            10.100.200.177       /* kube-system/kubernetes-dashboard: cluster IP */ tcp dpt:443
    0     0 KUBE-MARK-MASQ  tcp  --  *      *      !10.200.0.0/16        10.100.200.79        /* default/my-nginx: cluster IP */ tcp dpt:80
    0     0 KUBE-SVC-BEPXDJBUHFCSYIC3  tcp  --  *      *       0.0.0.0/0            10.100.200.79        /* default/my-nginx: cluster IP */ tcp dpt:80
    0     0 KUBE-MARK-MASQ  tcp  --  *      *      !10.200.0.0/16        10.100.200.1         /* default/kubernetes:https cluster IP */ tcp dpt:443
    0     0 KUBE-SVC-NPX46M4PTMTKRN6Y  tcp  --  *      *       0.0.0.0/0            10.100.200.1         /* default/kubernetes:https cluster IP */ tcp dpt:443
   15   888 KUBE-NODEPORTS  all  --  *      *       0.0.0.0/0            0.0.0.0/0            /* kubernetes service nodeports; NOTE: this must be the last rule in this chain */ ADDRTYPE match dst-type LOCAL

Chain KUBE-SVC-BEPXDJBUHFCSYIC3 (1 references)
 pkts bytes target     prot opt in     out     source               destination
    0     0 KUBE-SEP-AOFVCMQOP4BSPNKH  all  --  *      *       0.0.0.0/0            0.0.0.0/0            statistic mode random probability 0.50000000000
    0     0 KUBE-SEP-VAZHZYTNC3JEIHVH  all  --  *      *       0.0.0.0/0            0.0.0.0/0

Chain KUBE-SVC-ERIFXISQEP7F7OF4 (1 references)
 pkts bytes target     prot opt in     out     source               destination
    0     0 KUBE-SEP-7IVKYNOTO223MVHZ  all  --  *      *       0.0.0.0/0            0.0.0.0/0            statistic mode random probability 0.33332999982
    0     0 KUBE-SEP-MWWFI2FSQNNTZPVG  all  --  *      *       0.0.0.0/0            0.0.0.0/0            statistic mode random probability 0.50000000000
    0     0 KUBE-SEP-ORNXWSX5YDVWULHU  all  --  *      *       0.0.0.0/0            0.0.0.0/0

Chain KUBE-SVC-LC5QY66VUV2HJ6WZ (1 references)
 pkts bytes target     prot opt in     out     source               destination
    0     0 KUBE-SEP-FQ4F6LUEKDKKCOMF  all  --  *      *       0.0.0.0/0            0.0.0.0/0

Chain KUBE-SVC-NPX46M4PTMTKRN6Y (1 references)
 pkts bytes target     prot opt in     out     source               destination
    0     0 KUBE-SEP-O4OJPHL6SYITWREJ  all  --  *      *       0.0.0.0/0            0.0.0.0/0

Chain KUBE-SVC-TCOU7JCQXEZGVUNU (1 references)
 pkts bytes target     prot opt in     out     source               destination
    0     0 KUBE-SEP-2Q2G5JSM4U6DJ5CG  all  --  *      *       0.0.0.0/0            0.0.0.0/0            statistic mode random probability 0.33332999982
    0     0 KUBE-SEP-QVYN2TD7L6IA7X3K  all  --  *      *       0.0.0.0/0            0.0.0.0/0            statistic mode random probability 0.50000000000
    0     0 KUBE-SEP-JKMNWT3R5EVONNDL  all  --  *      *       0.0.0.0/0            0.0.0.0/0

Chain KUBE-SVC-XGLOHA7QRQ3V22RZ (2 references)
 pkts bytes target     prot opt in     out     source               destination
    0     0 KUBE-SEP-XXKY7ZZPXQPQ7VS6  all  --  *      *       0.0.0.0/0            0.0.0.0/0

Chain cali-OUTPUT (1 references)
 pkts bytes target     prot opt in     out     source               destination
23905 1445K cali-fip-dnat  all  --  *      *       0.0.0.0/0            0.0.0.0/0            /* cali:GBTAv2p5CwevEyJm */

Chain cali-POSTROUTING (1 references)
 pkts bytes target     prot opt in     out     source               destination
23928 1446K cali-fip-snat  all  --  *      *       0.0.0.0/0            0.0.0.0/0            /* cali:Z-c7XtVd2Bq7s_hA */
23928 1446K cali-nat-outgoing  all  --  *      *       0.0.0.0/0            0.0.0.0/0            /* cali:nYKhEzDlr11Jccal */
    0     0 MASQUERADE  all  --  *      tunl0   0.0.0.0/0            0.0.0.0/0            /* cali:SXWvdsbh4Mw7wOln */ ADDRTYPE match src-type !LOCAL limit-out ADDRTYPE match src-type LOCAL

Chain cali-PREROUTING (1 references)
 pkts bytes target     prot opt in     out     source               destination
 1268 73753 cali-fip-dnat  all  --  *      *       0.0.0.0/0            0.0.0.0/0            /* cali:r6XmIziWUJsdOK6Z */

Chain cali-fip-dnat (2 references)
 pkts bytes target     prot opt in     out     source               destination

Chain cali-fip-snat (1 references)
 pkts bytes target     prot opt in     out     source               destination

Chain cali-nat-outgoing (1 references)
 pkts bytes target     prot opt in     out     source               destination
   13   780 MASQUERADE  all  --  *      *       0.0.0.0/0            0.0.0.0/0            /* cali:flqWnvo8yq4ULQLa */ match-set cali40masq-ipam-pools src ! match-set cali40all-ipam-pools dst
```

Look in the KUBE-SERVICES chain above for your named service in a comment:

```
    0     0 KUBE-MARK-MASQ  tcp  --  *      *      !10.200.0.0/16        10.100.200.79        /* default/my-nginx: cluster IP */ tcp dpt:80
    0     0 KUBE-SVC-BEPXDJBUHFCSYIC3  tcp  --  *      *       0.0.0.0/0            10.100.200.79        /* default/my-nginx: cluster IP */ tcp dpt:80
```

This shows that the service IP is being IP masqueraded (SNAT) outbound if the inbound request is OUTSIDE of the network (!10.200.0.0/16), and then is being redirected (DNAT) via the KUBE-SVC-BEPXDJBUHFCSYIC3 chain.   Your chain will be a different random string in your terminal than the one above.   Follow that down the list to find the next entry with it:

```
Chain KUBE-SVC-BEPXDJBUHFCSYIC3 (1 references)
 pkts bytes target     prot opt in     out     source               destination
    0     0 KUBE-SEP-AOFVCMQOP4BSPNKH  all  --  *      *       0.0.0.0/0            0.0.0.0/0            statistic mode random probability 0.50000000000
    0     0 KUBE-SEP-VAZHZYTNC3JEIHVH  all  --  *      *       0.0.0.0/0            0.0.0.0/0
```

This shows ultimately what a service usually is, a randomized dispatch table to individual pods.  Let's hunt for the targets KUBE-SEP-AOFVCMQOP4BSPNKH and KUBE-SEP-VAZHZYTNC3JEIHVH.  Again, these will be a bit differently named in your terminal.

```
Chain KUBE-SEP-AOFVCMQOP4BSPNKH (1 references)
 pkts bytes target     prot opt in     out     source               destination
    0     0 KUBE-MARK-MASQ  all  --  *      *       10.200.165.70        0.0.0.0/0
    0     0 DNAT       tcp  --  *      *       0.0.0.0/0            0.0.0.0/0            tcp to:10.200.165.70:80
Chain KUBE-SEP-AOFVCMQOP4BSPNKH (1 references)
 pkts bytes target     prot opt in     out     source               destination
    0     0 KUBE-MARK-MASQ  all  --  *      *       10.200.165.70        0.0.0.0/0
    0     0 DNAT       tcp  --  *      *       0.0.0.0/0            0.0.0.0/0            tcp to:10.200.165.70:80
```

Finally, we see above the DNAT rule that routes the packet to the pod IPs, which whould match the podIPs you saw above.